In [10]:
!pip install -q tfds-nightly tensorflow-datasets tensorflow matplotlib

In [1]:
import tensorflow as tf
import cv2
import numpy as np
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import sklearn
import math

In [2]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [13]:
train_ds, validation_ds, test_ds = tfds.load('deep_weeds', as_supervised= True, shuffle_files= True, split=["train[:70%]", "train[:20%]", "train[:10%]"], batch_size = 32)

print("Number of training samples: %d" % tf.data.experimental.cardinality(train_ds))
print(
    "Number of validation samples: %d" % tf.data.experimental.cardinality(validation_ds)
)
print("Number of test samples: %d" % tf.data.experimental.cardinality(test_ds))

Number of training samples: 383
Number of validation samples: 110
Number of test samples: 55


In [6]:
from tensorflow.keras import datasets, layers, models, regularizers
import datetime

In [7]:
model = models.Sequential()

model.add(layers.Conv2D(96, (11, 11), activation='relu', input_shape=(256, 256, 3), strides = 4))
model.add(layers.MaxPooling2D((3, 3), strides = 2))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(256, (5, 5), activation='relu', padding = 'same'))
model.add(layers.MaxPooling2D((3, 3), strides = 2))

model.add(layers.Conv2D(192, (3, 3), activation='relu', padding = 'same'))
model.add(layers.Conv2D(192, (3, 3), activation='relu', padding = 'same'))
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding = 'same'))
model.add(layers.MaxPooling2D((3, 3), strides = 2))
model.add(layers.BatchNormalization())

model.add(layers.Flatten())

model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.25))

model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.25))

model.add(layers.Dense(9))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 96)        34944     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 30, 30, 96)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 30, 30, 96)        384       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 256)       614656    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 192)       442560    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 192)       3

In [14]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])




model.fit(train_ds, epochs=50, 
                    validation_data=validation_ds)

Epoch 1/50
383/383 [==============================] - 36s 93ms/step - loss: 0.7265 - accuracy: 0.7427 - val_loss: 1.8973 - val_accuracy: 0.5725
Epoch 2/50
383/383 [==============================] - 35s 90ms/step - loss: 0.5621 - accuracy: 0.8012 - val_loss: 1.3420 - val_accuracy: 0.6865
Epoch 3/50
383/383 [==============================] - 35s 91ms/step - loss: 0.4929 - accuracy: 0.8267 - val_loss: 1.6842 - val_accuracy: 0.6773
Epoch 4/50
383/383 [==============================] - 35s 91ms/step - loss: 0.4303 - accuracy: 0.8441 - val_loss: 1.0516 - val_accuracy: 0.7493
Epoch 5/50
383/383 [==============================] - 35s 90ms/step - loss: 0.4172 - accuracy: 0.8580 - val_loss: 1.2857 - val_accuracy: 0.6828
Epoch 6/50
383/383 [==============================] - 35s 90ms/step - loss: 0.3549 - accuracy: 0.8764 - val_loss: 1.1070 - val_accuracy: 0.7481
Epoch 7/50
383/383 [==============================] - 35s 91ms/step - loss: 0.3400 - accuracy: 0.8840 - val_loss: 0.6980 - val_accuracy:

In [25]:
y_pred = model.predict(test_ds)
y_pred = np.argmax(y_pred, axis=1) # Convert one-hot to index

In [15]:
test = tfds.as_numpy(test_ds)
y_pred = []
y_label = []

for i in test:
  flat_img = i[0]
  flat_label = i[1]
  pred = model.predict(flat_img)
  pred = np.argmax(pred, axis=1)
  y_pred.extend(pred)
  y_label.extend(flat_label)

y_pred = np.array(y_pred)
y_label = np.array(y_label)

In [17]:
from sklearn.metrics import classification_report

print(y_pred)
print(classification_report(y_label, y_pred))

[1 7 7 ... 8 8 0]
              precision    recall  f1-score   support

           0       0.99      0.92      0.95        98
           1       0.91      0.97      0.94       108
           2       0.93      0.98      0.96       101
           3       0.91      0.99      0.95       114
           4       0.96      1.00      0.98        91
           5       1.00      0.73      0.85       109
           6       0.81      1.00      0.89       105
           7       0.97      0.94      0.95        98
           8       0.98      0.97      0.98       927

    accuracy                           0.96      1751
   macro avg       0.94      0.95      0.94      1751
weighted avg       0.96      0.96      0.96      1751

